In [1]:
import pandas as pd

pd.set_option("display.max_columns", None)

import cufflinks as cf

# We set the all charts as public
cf.set_config_file(sharing="public", theme="pearl", offline=False)
cf.go_offline()

In [2]:
# read data
price_df = pd.read_csv("./data/forecasting.csv")
price_df.columns = price_df.columns.str.lower()
price_df["date"] = pd.to_datetime(price_df["date"])
len(price_df)

43544

## Feature Engineering

In [3]:
# engineer some features
price_df["is_apartment"] = (
    price_df["categories"].map(lambda x: "APARTMENT" in x).map(int)
)
price_df["categories"] = price_df["categories"].str.replace("APARTMENT, ", "")

price_df["is_duplex"] = price_df["categories"].map(lambda x: "DUPLEX" in x).map(int)
price_df["categories"] = price_df["categories"].str.replace("DUPLEX, ", "")


price_df["is_house"] = price_df["categories"].map(lambda x: "HOUSE" in x).map(int)
price_df["categories"] = price_df["categories"].str.replace("HOUSE, ", "")

d = {range(0, 50): "sm", range(50, 100): "md", range(100, 2000): "bg"}

price_df["size"] = price_df["space"].apply(
    lambda x: next((v for k, v in d.items() if x in k), "sm")
)

# categorize with ChatGPT
residential = [
    "DUPLEX",
    "FURNISHED_FLAT",
    "APARTMENT",
    "FLAT",
    "ROOF_FLAT",
    "ATTIC_FLAT",
    "LOFT",
    "SINGLE_ROOM",
    "STUDIO",
    "HOUSE",
    "ROW_HOUSE",
    "DUPLEX, MAISONETTE",
    "HOUSE, SINGLE_HOUSE",
    "ROW_HOUSE",
    "ATTIC",
    "BIFAMILIAR_HOUSE",
    "HOUSE, BIFAMILIAR_HOUSE",
    "TERRACE_FLAT",
    "BACHELOR_FLAT",
    "HOUSE, VILLA",
    "HOUSE, MULTIPLE_DWELLING",
    "VILLA",
]

commercial = [
    "OFFICE",
    "COMMERCIAL",
    "RETAIL, PRACTICE",
    "SHOP, ATELIER",
    "RETAIL, COMMERCIAL",
    "RESTAURANT",
    "WORKSHOP",
    "RETAIL, SHOP",
    "SHOP",
    "RESIDENTIAL_COMMERCIAL_BUILDING",
    "HOBBY_ROOM",
    "INDUSTRIAL_OBJECT",
    "STORAGE_ROOM",
    "ATELIER",
    "CAFE_BAR, RESTAURANT",
    "RETAIL, HAIRDRESSER",
    "RETAIL_SPACE, ATELIER",
    "ADVERTISING_AREA",
    "FACTORY",
    "RETAIL, STORAGE_ROOM",
    "UNDERGROUND_SLOT",
    "CAFE_BAR, BAR",
    "PRACTICE",
    "COFFEEHOUSE",
]

price_df["is_residential"] = price_df["categories"].isin(residential).map(int).fillna(0)
price_df["is_commercial"] = price_df["categories"].isin(commercial).map(int).fillna(0)

In [4]:
# remove NAs in target
print(price_df.price.isna().value_counts())
print(price_df.space.isna().value_counts())
price_df = price_df.dropna(subset=["price"])  # "space"
len(price_df)

False    37397
True      6147
Name: price, dtype: int64
False    34545
True      8999
Name: space, dtype: int64


37397

In [5]:
price_df.drop(columns=["date", "categories"]).corr().style.background_gradient(
    cmap="coolwarm"
)

/var/folders/8y/630_g50j6sg5c2q1qypy80pw0000gn/T/ipykernel_46620/4014027835.py:1: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



,price,space,is_apartment,is_duplex,is_house,is_residential,is_commercial
price,1.000000,0.159076,-0.042458,0.009543,0.018420,-0.044072,0.046681
space,0.159076,1.000000,-0.187060,0.247235,0.270212,-0.263992,0.154457
is_apartment,-0.042458,-0.187060,1.000000,-0.144783,-0.194864,0.711457,-0.660235
is_duplex,0.009543,0.247235,-0.144783,1.000000,-0.021266,0.027209,-0.072052
is_house,0.018420,0.270212,-0.194864,-0.021266,1.000000,-0.099882,-0.041227
is_residential,-0.044072,-0.263992,0.711457,0.027209,-0.099882,1.000000,-0.900368
is_commercial,0.046681,0.154457,-0.660235,-0.072052,-0.041227,-0.900368,1.000000


In [6]:
residential_df = price_df.query("is_residential == 1").drop(
    columns=["is_commercial", "is_residential"]
)
len(residential_df)

31582

In [7]:
residential_df.sort_values("is_apartment")

,date,price,space,categories,is_apartment,is_duplex,is_house,size
0,2018-04-07,2650.0,190.0,DUPLEX,0,1,0,bg
36571,2018-10-25,3700.0,130.0,ROW_HOUSE,0,0,1,bg
28897,2019-06-12,1050.0,35.0,FURNISHED_FLAT,0,0,0,sm
36576,2018-04-03,1400.0,85.0,ROOF_FLAT,0,0,0,md
36585,2018-05-15,799.0,25.0,FURNISHED_FLAT,0,0,0,sm
...,...,...,...,...,...,...,...,...
15013,2021-01-22,1700.0,91.0,ROOF_FLAT,1,0,0,md
15011,2021-08-11,1900.0,70.0,FLAT,1,0,0,md
15010,2021-02-05,1400.0,75.0,FLAT,1,0,0,md
15024,2021-06-16,700.0,34.0,STUDIO,1,0,0,sm


In [8]:
residential_df.join(pd.get_dummies(residential_df["categories"]).applymap(int)).drop(
    columns=["date", "categories"]
).corr().style.background_gradient(cmap="coolwarm")

/var/folders/8y/630_g50j6sg5c2q1qypy80pw0000gn/T/ipykernel_46620/4052620087.py:3: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



,price,space,is_apartment,is_duplex,is_house,APARTMENT,ATTIC,ATTIC_FLAT,BACHELOR_FLAT,BIFAMILIAR_HOUSE,DUPLEX,FLAT,FURNISHED_FLAT,LOFT,ROOF_FLAT,ROW_HOUSE,SINGLE_ROOM,STUDIO,TERRACE_FLAT,VILLA
price,1.000000,0.130818,-0.040999,0.032242,0.035898,-0.027656,-0.012218,0.016550,-0.000231,0.017905,0.032242,-0.018159,0.052532,0.005107,0.006134,0.028452,-0.024823,-0.013890,0.001884,0.021388
space,0.130818,1.000000,-0.129729,0.287626,0.202538,-0.046219,-0.079560,0.077122,-0.004281,0.131518,0.287626,-0.071248,0.009600,0.048819,0.078471,0.155293,-0.223986,-0.127795,0.012588,0.094406
is_apartment,-0.040999,-0.129729,1.000000,-0.317157,-0.241076,0.304123,-0.066672,-0.138909,-0.012999,-0.108945,-0.317157,0.295703,-0.372489,-0.088829,-0.241861,-0.209258,-0.176435,-0.134332,-0.028819,-0.067531
is_duplex,0.032242,0.287626,-0.317157,1.000000,-0.017295,-0.157018,-0.009264,-0.021853,-0.002166,-0.007816,1.000000,-0.152670,-0.059082,-0.013905,-0.036903,-0.015012,-0.025787,-0.019005,-0.005630,-0.004845
is_house,0.035898,0.202538,-0.241076,-0.017295,1.000000,-0.073317,-0.004325,-0.010204,-0.001011,0.451911,-0.017295,-0.071287,-0.027587,-0.006493,-0.017231,0.868015,-0.012041,-0.008874,-0.002629,0.138929
APARTMENT,-0.027656,-0.046219,0.304123,-0.157018,-0.073317,1.000000,-0.039270,-0.092638,-0.009182,-0.033133,-0.157018,-0.647201,-0.250459,-0.058945,-0.156439,-0.063640,-0.109317,-0.080566,-0.023865,-0.020538
ATTIC,-0.012218,-0.079560,-0.066672,-0.009264,-0.004325,-0.039270,1.000000,-0.005465,-0.000542,-0.001955,-0.009264,-0.038183,-0.014776,-0.003478,-0.009229,-0.003755,-0.006449,-0.004753,-0.001408,-0.001212
ATTIC_FLAT,0.016550,0.077122,-0.138909,-0.021853,-0.010204,-0.092638,-0.005465,1.000000,-0.001278,-0.004611,-0.021853,-0.090074,-0.034857,-0.008204,-0.021772,-0.008857,-0.015214,-0.011213,-0.003321,-0.002858
BACHELOR_FLAT,-0.000231,-0.004281,-0.012999,-0.002166,-0.001011,-0.009182,-0.000542,-0.001278,1.000000,-0.000457,-0.002166,-0.008928,-0.003455,-0.000813,-0.002158,-0.000878,-0.001508,-0.001111,-0.000329,-0.000283
BIFAMILIAR_HOUSE,0.017905,0.131518,-0.108945,-0.007816,0.451911,-0.033133,-0.001955,-0.004611,-0.000457,1.000000,-0.007816,-0.032215,-0.012467,-0.002934,-0.007787,-0.003168,-0.005441,-0.004010,-0.001188,-0.001022


In [9]:
residential_df

,date,price,space,categories,is_apartment,is_duplex,is_house,size
0,2018-04-07,2650.0,190.0,DUPLEX,0,1,0,bg
1,2018-03-15,2850.0,190.0,DUPLEX,0,1,0,bg
2,2020-06-05,4600.0,170.0,FURNISHED_FLAT,1,0,0,bg
6,2018-03-13,1370.0,NaN,APARTMENT,1,0,0,sm
8,2018-11-06,1110.0,NaN,APARTMENT,1,0,0,sm
...,...,...,...,...,...,...,...,...
43537,2021-02-04,1320.0,75.0,FLAT,1,0,0,md
43538,2020-08-10,1950.0,120.0,LOFT,1,0,0,bg
43539,2020-08-14,1990.0,70.0,DUPLEX,1,1,0,md
43542,2020-08-18,1540.0,75.0,FLAT,1,0,0,md


In [39]:
residential_df["space"].iplot(kind="hist")

## Modelling

In [26]:
from pycaret.regression import RegressionExperiment

s = RegressionExperiment()
s.setup(
    residential_df.drop(columns=["date"]),
    target="price",
    session_id=123,
    log_experiment=True,
    experiment_name="price_forecast",
)

,Description,Value
0,Session id,123
1,Target,price
2,Target type,Regression
3,Original data shape,"(31582, 7)"
4,Transformed data shape,"(31582, 23)"
5,Transformed train set shape,"(22107, 23)"
6,Transformed test set shape,"(9475, 23)"
7,Numeric features,4
8,Categorical features,2
9,Rows with missing values,4.5%


run `mlflow ui` in the terminal

In [27]:
best = s.compare_models(
    include=[
        "lr",
        "ridge",
        "lasso",
        "huber",
        "rf",
        "catboost",
        "lightgbm",
        "knn",
        "xgboost",
    ]
)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
knn,K Neighbors Regressor,329.1667,34414661.7531,2312.0967,0.5987,0.2621,0.2253,0.0470
lr,Linear Regression,342.5561,34430194.6191,2322.8745,0.5848,0.2743,0.2260,0.0230
ridge,Ridge Regression,342.7087,34430319.5985,2323.0038,0.5846,0.2743,0.2261,0.0170
huber,Huber Regressor,326.6719,34432323.1634,2323.5841,0.5841,0.2676,0.2073,0.0800
lasso,Lasso Regression,344.7067,34433187.9500,2325.2129,0.5818,0.2766,0.2284,0.0250
lightgbm,Light Gradient Boosting Machine,346.0039,34831136.9671,2614.0815,0.0732,0.2839,0.2549,0.4300
catboost,CatBoost Regressor,343.9409,34839509.5402,2618.3184,0.0640,0.2827,0.2542,0.4310
xgboost,Extreme Gradient Boosting,342.8062,34844786.4500,2621.4472,0.0570,0.2821,0.2469,0.0360
rf,Random Forest Regressor,344.0800,34888610.1600,2645.6300,-0.0068,0.2832,0.2351,0.2060


In [28]:
print(best)

KNeighborsRegressor(n_jobs=-1)


In [29]:
s.evaluate_model(best)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [35]:
mdl = s.create_model("knn")
tuned_model = s.tune_model(mdl, optimize="RMSE", search_library="scikit-optimize")
s.evaluate_model(tuned_model)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,287.0438,238713.9219,488.5836,0.6875,0.2494,0.1942
1,286.0844,257937.2500,507.8752,0.6676,0.2793,0.4883
2,301.8379,596321.7500,772.2187,0.4790,0.2746,0.2030
3,690.3418,341676224.0000,18484.4863,0.0003,0.2941,0.1931
4,286.3221,216588.2656,465.3904,0.6642,0.2579,0.1987
5,294.3958,234682.3750,484.4403,0.6976,0.2724,0.2121
6,275.8982,210413.9688,458.7090,0.6991,0.2548,0.1959
7,282.3224,197847.6719,444.8007,0.7132,0.2414,0.1866
8,297.0919,300282.7188,547.9806,0.6700,0.2490,0.1893


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,279.6194,240670.8880,490.5822,0.6849,0.2484,0.1909
1,280.2260,268524.2002,518.1932,0.6539,0.2859,0.4858
2,287.5806,589434.9098,767.7466,0.4850,0.2649,0.1924
3,677.8040,341697534.2801,18485.0625,0.0002,0.2836,0.1841
4,273.4786,200223.7250,447.4637,0.6896,0.2512,0.1920
5,281.8715,224517.3935,473.8327,0.7107,0.2619,0.2016
6,264.4256,191717.6438,437.8557,0.7258,0.2497,0.1897
7,270.1776,187900.1480,433.4745,0.7276,0.2350,0.1799
8,273.8691,267157.7470,516.8730,0.7064,0.2327,0.1740


Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [37]:
s.predict_model(tuned_model)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,K Neighbors Regressor,270.4185,209733.6010,457.9668,0.7230,0.2343,0.1753


,space,categories,is_apartment,is_duplex,is_house,size,price,prediction_label
34346,60.0,FLAT,1,0,0,md,1220.0,1207.4
30106,105.0,APARTMENT,1,0,0,bg,2790.0,2028.0
28664,128.0,ATTIC_FLAT,0,0,0,bg,2880.0,2880.0
39871,158.0,DUPLEX,0,1,0,bg,2650.0,2650.0
35500,81.0,APARTMENT,1,0,0,md,1450.0,1492.5
...,...,...,...,...,...,...,...,...
1404,86.0,FLAT,1,0,0,md,1950.0,1792.0
40436,31.0,FLAT,1,0,0,sm,700.0,752.0
40129,35.0,FLAT,1,0,0,sm,1000.0,776.5
26537,NaN,APARTMENT,1,0,0,sm,1220.0,1602.0


In [38]:
s.predict_model(best)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,K Neighbors Regressor,285.7596,231049.0312,480.6756,0.6949,0.2441,0.1849


,space,categories,is_apartment,is_duplex,is_house,size,price,prediction_label
34346,60.0,FLAT,1,0,0,md,1220.0,1222.800049
30106,105.0,APARTMENT,1,0,0,bg,2790.0,2018.000000
28664,128.0,ATTIC_FLAT,0,0,0,bg,2880.0,3415.000000
39871,158.0,DUPLEX,0,1,0,bg,2650.0,2856.000000
35500,81.0,APARTMENT,1,0,0,md,1450.0,1489.000000
...,...,...,...,...,...,...,...,...
1404,86.0,FLAT,1,0,0,md,1950.0,1722.000000
40436,31.0,FLAT,1,0,0,sm,700.0,775.000000
40129,35.0,FLAT,1,0,0,sm,1000.0,783.000000
26537,NaN,APARTMENT,1,0,0,sm,1220.0,1702.000000


## Model Price per square meter

In [19]:
ts_df = residential_df.dropna(subset=["space"]).reset_index(drop=True)
ts_df["price_sqm"] = ts_df.eval("price/space").map(lambda x: int(x * 100) / 100)
ts_df = ts_df.join(pd.get_dummies(ts_df["categories"]).applymap(int))
ts_df.shape

(30148, 24)

In [20]:
ts_df.set_index("date")["price_sqm"].resample("M").mean().iplot()

In [21]:
ts_df = ts_df.sort_values("price_sqm").query("price_sqm<=200").reset_index(drop=True)

In [22]:
ts_df.set_index("date")["price_sqm"].resample("M").mean().iplot()

In [23]:
ts_df.query("is_duplex==1").groupby(pd.to_datetime(ts_df.date.dt.strftime("%Y-%m-01")))[
    "price_sqm"
].mean().iplot()

In [ ]:
from pycaret.regression import RegressionExperiment

s = RegressionExperiment()
s.setup(
    residential_df.drop(columns=["date"]),
    target="price_sqm",
    session_id=123,
    log_experiment=True,
    experiment_name="price_forecast",
)